In [ ]:
# Version 8
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import Birch
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import uniform, loguniform


# Load the data
def load_data(file_path):
    try:
        df = pd.read_csv(file_path)
        df["Date"] = pd.to_datetime(df["Date"])
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None


# Function to extract time series features
def extract_time_series_features(group):
    return pd.Series(
        {
            "Age": group["Age"].iloc[0],
            "Sex": group["Sex"].iloc[0],
            "Smoke": group["Smoke"].iloc[0],
            "Smoke_amount": group["Smoke_amount"].iloc[0],
            "Height": group["Height"].iloc[0],
            "Weight": group["Weight"].iloc[0],
            "BMI": group["BMI"].iloc[0],
            "BSA": group["BSA"].iloc[0],
            "Morning_PEFR_mean": group["Morning_PEFR"].mean(),
            "Morning_PEFR_std": group["Morning_PEFR"].std(),
            "Afternoon_PEFR_mean": group["Afternoon_PEFR"].mean(),
            "Afternoon_PEFR_std": group["Afternoon_PEFR"].std(),
            "PEFR_range": (group["Afternoon_PEFR"] - group["Morning_PEFR"]).mean(),
        }
    )


# Preprocess the data
def preprocess_data(df):

    # Convert categorical columns to numeric
    df["Sex"] = df["Sex"].map({"M": 0, "F": 1})
    df["Smoke"] = df["Smoke"].map({"NS": 0, "ES": 1, "SM": 2})

    # Apply log transformation to handle potential extreme values
    numeric_cols = [
        "Age",
        "Smoke_amount",
        "Height",
        "Weight",
        "BMI",
        "BSA",
        "Morning_PEFR",
        "Afternoon_PEFR",
    ]
    for col in numeric_cols:
        df[col] = np.log1p(df[col])

    return df.groupby("ID").apply(extract_time_series_features).reset_index()


# Define features for clustering
numeric_features = [
    "Age",
    "Smoke_amount",
    "Height",
    "Weight",
    "BMI",
    "BSA",
    "Morning_PEFR_mean",
    "Morning_PEFR_std",
    "Afternoon_PEFR_mean",
    "Afternoon_PEFR_std",
    "PEFR_range",
]
categorical_features = ["Sex", "Smoke"]

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(drop="first", sparse_output=False), categorical_features),
    ],
    force_int_remainder_cols=False,
)

# Create the BIRCH model
birch = Birch(n_clusters=None)


# Create BIRCH clustering pipeline
birch_pipeline = Pipeline([("preprocessor", preprocessor), ("birch", birch)])


# Define a comprehensive scoring function
def clustering_score(estimator, X):
    preprocessed_X = estimator.named_steps["preprocessor"].transform(X)
    labels = estimator.named_steps["birch"].fit_predict(preprocessed_X)
    n_clusters = len(np.unique(labels))

    if n_clusters == 1:
        return 0  # Return 0 instead of -np.inf for single cluster case

    try:
        sil_score = silhouette_score(preprocessed_X, labels)
        ch_score = calinski_harabasz_score(preprocessed_X, labels)
        return (0.7 * sil_score) + (0.3 * ch_score)
    except Exception as e:
        print(f"Error in clustering_score: {e}")
        return 0  # Return 0 instead of -np.inf


# Define parameter grid for BIRCH
param_grid = {
    "birch__threshold": uniform(0.1, 2.0),
    "birch__branching_factor": np.arange(10, 100),
    "birch__n_clusters": [None] + list(range(2, 21)),  # None and 2 to 20 clusters
}


# liming clusters to max 10 or as mentioned
def limit_clusters(labels, max_clusters=10):
    unique_labels = np.unique(labels)
    if len(unique_labels) > max_clusters:
        # Merge smaller clusters
        label_counts = np.bincount(labels)
        top_labels = np.argsort(label_counts)[-max_clusters:]
        new_labels = np.zeros_like(labels)
        for i, label in enumerate(top_labels):
            new_labels[labels == label] = i
        return new_labels
    return labels


# Perform grid search
def perform_grid_search(birch_pipeline, param_grid, X):

    # First, use RandomizedSearchCV to explore a wide range of parameters
    random_search = RandomizedSearchCV(
        birch_pipeline,
        param_distributions=param_grid,
        n_iter=100,  # Number of parameter settings sampled
        scoring=clustering_score,
        n_jobs=-1,
        cv=5,
        verbose=2,
        random_state=42,
    )

    # Fit RandomizedSearchCV
    random_search.fit(X)  # X is your preprocessed data

    # Get the best parameters from RandomizedSearchCV
    best_params = random_search.best_params_

    # Define a focused parameter grid based on the best results from RandomizedSearchCV
    focused_param_grid = {
        "birch__branching_factor": range(
            max(10, best_params.get("branching_factor", 50) - 10),
            min(100, best_params.get("branching_factor", 50) + 11),
            5,
        ),
        "birch__n_clusters": [None]
        + list(
            range(
                max(
                    2,
                    (
                        best_params.get("n_clusters", 5) - 2
                        if best_params.get("n_clusters")
                        else 2
                    ),
                ),
                min(
                    20,
                    (
                        best_params.get("n_clusters", 5) + 3
                        if best_params.get("n_clusters")
                        else 5
                    ),
                ),
            )
        ),
    }

    grid_search = GridSearchCV(
        birch_pipeline,
        param_grid=focused_param_grid,
        scoring=clustering_score,
        cv=5,
        n_jobs=-1,
        verbose=2,
    )
    grid_search.fit(X)

    # Get the best model and limit the number of clusters
    best_model = grid_search.best_estimator_
    labels = best_model.named_steps["birch"].labels_
    limited_labels = limit_clusters(labels, 20)
    best_model.named_steps["birch"].labels_ = limited_labels

    # Use the best model for final clustering
    final_labels = best_model.fit_predict(X)

    print(f"final labels : {final_labels}")

    return grid_search, best_model


# Visualize clusters using PCA
def visualize_clusters(X, labels, title):
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(X)
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], c=labels, cmap="viridis")
    plt.colorbar(scatter)
    plt.title(title)
    plt.xlabel("First Principal Component")
    plt.ylabel("Second Principal Component")
    plt.show()


# Analyze feature importance
def analyze_feature_importance(feature_names, subcluster_centers):
    importances = np.abs(subcluster_centers).mean(axis=0)

    # Ensure feature_names and importances have the same length
    if len(feature_names) != len(importances):
        print(
            f"Warning: Mismatch in feature names ({len(feature_names)}) and importance scores ({len(importances)})"
        )
        # Truncate the longer list to match the shorter one
        min_length = min(len(feature_names), len(importances))
        feature_names = feature_names[:min_length]
        importances = importances[:min_length]

    feature_importance = pd.DataFrame(
        {"feature": feature_names, "importance": importances}
    )
    feature_importance = feature_importance.sort_values("importance", ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(x="importance", y="feature", data=feature_importance)
    plt.title("Feature Importance in Clustering")
    plt.tight_layout()
    plt.show()

    return feature_importance


# Print cluster characteristics
def print_cluster_characteristics(df, labels):
    df_with_clusters = df.copy()
    df_with_clusters["Cluster"] = labels
    for cluster in range(len(np.unique(labels))):
        print(f"\nCluster {cluster} characteristics:")
        cluster_data = df_with_clusters[df_with_clusters["Cluster"] == cluster]
        for feature in numeric_features + categorical_features:
            if feature in numeric_features:
                print(
                    f"{feature}: mean = {cluster_data[feature].mean():.2f}, std = {cluster_data[feature].std():.2f}"
                )
            else:
                print(f"{feature}:")
                print(cluster_data[feature].value_counts(normalize=True))
        print("-" * 40)


# Main
def main():
    # Load and preprocess data
    df = load_data("2024-Data-Cleaned/merged_patient_data.csv")
    if df is None:
        return

    df_features = preprocess_data(df)

    # Perform grid search
    print("Performing BIRCH clustering grid search...")
    grid_search, best_model = perform_grid_search(
        birch_pipeline, param_grid, df_features
    )

    # Print best parameters and score
    print("Best parameters:", grid_search.best_params_)
    print("Best score:", grid_search.best_score_)

    # Get cluster labels
    cluster_labels = best_model.named_steps["birch"].labels_

    # Check for single cluster case
    if len(np.unique(cluster_labels)) < 2:
        print(
            "Warning: Only one cluster found. Clustering may not be effective for this data."
        )
        return

    # Print cluster sizes
    print("\nCluster sizes:")
    print(pd.Series(cluster_labels).value_counts())

    # Visualize clusters
    preprocessed_X = best_model.named_steps["preprocessor"].transform(df_features)
    visualize_clusters(
        preprocessed_X, cluster_labels, "Patient Clusters Visualization (PCA)"
    )

    # Get feature names after preprocessing
    numeric_transformer = best_model.named_steps["preprocessor"].named_transformers_[
        "num"
    ]
    categorical_transformer = best_model.named_steps[
        "preprocessor"
    ].named_transformers_["cat"]
    feature_names = (
        numeric_features
        + best_model.named_steps["preprocessor"]
        .named_transformers_["cat"]
        .get_feature_names_out(categorical_features)
        .tolist()
    )

    subcluster_centers = best_model.named_steps["birch"].subcluster_centers_
    feature_importance = analyze_feature_importance(feature_names, subcluster_centers)

    # Print cluster characteristics
    print_cluster_characteristics(df_features, cluster_labels)

    # Save results
    np.save("Results/cluster_birch_labels.npy", cluster_labels)
    feature_importance.to_csv(
        "Results/cluster_birch_feature_importance.csv", index=False
    )
    print("Results saved to cluster_labels.npy and feature_importance.csv")


if __name__ == "__main__":
    main()